In [ ]:
import pandas as pd
import os

from transformers import WhisperForConditionalGeneration
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from transformers import WhisperFeatureExtractor
import torch
from transformers import  pipeline

import json
import evaluate

c:\Users\harsh\.virtualenvs\CollectWise-7aTbe7cb\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Processed data stats

In [19]:
import os
import librosa

# Path to processed files
processed_path = os.path.join(ROOT_FOLDER, "data", "Processed_Files")

total_duration = 0.0
file_durations = {}

# Loop through all files
for fname in os.listdir(processed_path):
    fpath = os.path.join(processed_path, fname)
    
    if os.path.isfile(fpath) and fname.lower().endswith((".wav", ".mp3", ".flac", ".ogg")):
        try:
            duration = librosa.get_duration(filename=fpath)
            file_durations[fname] = duration
            total_duration += duration
        except Exception as e:
            print(f"Skipping {fname}, error: {e}")

# Convert to hours
total_hours = total_duration / 3600

print(f"✅ Total audio duration: {total_hours:.2f} hours")
print(f"✅ Number of files: {len(file_durations)}")


C:\Users\harsh\AppData\Local\Temp\ipykernel_7432\2283642796.py:16: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=fpath)


✅ Total audio duration: 14.46 hours
✅ Number of files: 1626


## Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
import os
import pandas as pd

ROOT_FOLDER = ''

processed_path = os.path.join(ROOT_FOLDER, "data", "Processed_Files")
all_files = [f for f in os.listdir(processed_path) if f != 'Transcript.json']

# First split: Train (85%) vs Temp (15%)
train_files, temp_files = train_test_split(all_files, test_size=0.15, random_state=42)

# Second split: Temp (15%) into Validation (7.5%) and Test (7.5%)
val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

# Build DataFrames
df_train = pd.DataFrame({"filename": train_files, "split": "train"})
df_val   = pd.DataFrame({"filename": val_files, "split": "val"})
df_test  = pd.DataFrame({"filename": test_files, "split": "test"})

# Combine all
df = pd.concat([df_train, df_val, df_test], ignore_index=True)

# Save CSV in the same folder
output_csv = os.path.join(processed_path, "file_splits.csv")
df.to_csv(output_csv, index=False)

print(df["split"].value_counts(normalize=True))


split
train    0.849416
test     0.075599
val      0.074985
Name: proportion, dtype: float64


## Pretrained whisper-small Test validation res

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

# Read JSON file
ROOT_FOLDER = '.'
with open(os.path.join(ROOT_FOLDER, 'data/Processed_Files', "Transcript.json"), "r", encoding="utf-8") as f:
    transcript_json = json.load(f)


In [5]:
from Model.Validation import validation

In [16]:
file_list = ['20241208090446_9873656524.mp3_lc.wav']
res = validation(model , processor, transcript_json, file_list, ROOT_FOLDER = "data/Processed_Files/" )

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/2868